In [1]:
%reset -f
import glob, os, sys, io
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
import keras

from wavhandler import *
from utils import *

import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)
import seaborn as sns
sns.set()

Using TensorFlow backend.


In [2]:
X,y,filenames = get_data(target_names=all_6, nr_signals=20000, only_names=False)

100%|██████████| 6/6 [00:44<00:00,  7.49s/it]


In [4]:
X = transform_data(X)

100%|██████████| 120906/120906 [01:09<00:00, 1731.06it/s]


In [5]:
X_train, X_test, y_train, y_test, files_train, files_test = train_test_split(X, y, filenames,
                                                                            test_size=0.20, 
                                                                            shuffle=True, 
                                                                            random_state=0)

In [9]:
# Convert label to onehot
y_train = keras.utils.to_categorical(y_train, num_classes=6)
y_test = keras.utils.to_categorical(y_test, num_classes=6)

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(4097, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model_name = 'deep_1_PSD'
top_weights_path = './data/model_' + str(model_name) + '.h5'

In [ ]:
callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', 
                                  verbose = 1, 
                                  save_best_only = True, 
                                  save_weights_only = True),
                EarlyStopping(monitor = 'val_acc', 
                              patience = 6, 
                              verbose = 1),
                ReduceLROnPlateau(monitor = 'val_acc', 
                                  factor = 0.1, 
                                  patience = 3, 
                                  verbose = 1),
                CSVLogger('model_' + str(model_name) + '.log')]

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_test, y_test], callbacks = callbacks_list)


In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)

#print('loss', loss)
print('Test accuracy:', acc)